# MNEMOS 2.1 – Institutional-Grade Market Intelligence (Colab)

**Runs 24/7 on free Colab.** NSE stocks + Nifty, Sensex, Bank Nifty, Nifty IT + Gold/Silver. Friction + GROQ Llama analysis, daily heartbeat, reports. Mail trail: comma-separated ALERT_EMAIL_TO.

1. **Upload** the project (zip or clone) and set `MNEMOS_ROOT` below.
2. **Secrets**: Add in Colab Secrets (key icon in sidebar) or paste in Section 2 cell: TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID, GMAIL_USER, GMAIL_APP_PASSWORD, ALERT_EMAIL_TO, GROQ_API_KEY. See cell below for step-by-step.
3. **Run all** – last cell runs MNEMOS forever. Use **--no-supervisor** so Colab doesn’t restart the process; re-run the cell if session drops.

**Keepalive**: Keep tab active; optional: add a cell with `%%javascript` and run every ~60 min:
`document.querySelector('.colab-connect-button')?.click();`

### How to add secrets (all secure – never in the repo)

**Option A – Colab Secrets (recommended)**  
1. Click the **key icon** in the left sidebar (Secrets).  
2. Click "Add new secret" and add each of these **names** with your **value**:

- **TELEGRAM_BOT_TOKEN** – from @BotFather on Telegram  
- **TELEGRAM_CHAT_ID** – from @userinfobot (your numeric ID)  
- **GMAIL_USER** – your Gmail address  
- **GMAIL_APP_PASSWORD** – Google Account → Security → 2-Step Verification → App passwords (16 characters, not your login password)  
- **ALERT_EMAIL_TO** – same as GMAIL_USER, or comma-separated: `a@gmail.com,b@gmail.com`  
- **GROQ_API_KEY** – from console.groq.com (free tier)

The code in Section 2 reads these automatically. Do not paste secrets in the notebook if you use Secrets.

**Option B – Paste in Section 2 cell**  
In the next code cell (Section 2), find the line that says `# PASTE YOUR VALUES BELOW` and replace the empty strings with your values. Do not share the notebook after pasting.

## 1. Install dependencies

In [ ]:
!pip install -q yfinance pandas numpy feedparser requests python-dotenv schedule tenacity pytz psutil
!pip install -q python-telegram-bot
print('Dependencies installed.')

## 2. Set project path and secrets

In [ ]:
import os
from pathlib import Path

# When opened from GitHub (Open in Colab): repo is usually current dir or /content/Mnemos-2.0
MNEMOS_ROOT = Path('.')
if not (MNEMOS_ROOT / 'main.py').exists():
    MNEMOS_ROOT = Path('/content/Mnemos-2.0')
if not MNEMOS_ROOT.exists() or not (MNEMOS_ROOT / 'main.py').exists():
    MNEMOS_ROOT = Path('/content/mnemos2.0')
if not MNEMOS_ROOT.exists():
    MNEMOS_ROOT = Path('.')
os.chdir(MNEMOS_ROOT)
# Load .env from project if present (e.g. you zipped the project with .env)
from dotenv import load_dotenv
load_dotenv(MNEMOS_ROOT / '.env')
print('Working dir:', os.getcwd())

# ----- Colab Secrets (key icon in sidebar) are read automatically. Or paste your values in the quotes below -----
os.environ.setdefault('TELEGRAM_BOT_TOKEN', os.environ.get('TELEGRAM_BOT_TOKEN', ''))
os.environ.setdefault('TELEGRAM_CHAT_ID', os.environ.get('TELEGRAM_CHAT_ID', ''))
os.environ.setdefault('GMAIL_USER', os.environ.get('GMAIL_USER', ''))
os.environ.setdefault('GMAIL_APP_PASSWORD', os.environ.get('GMAIL_APP_PASSWORD', ''))
# One mail trail: comma-separated addresses (e.g. a@gmail.com,b@gmail.com)
os.environ.setdefault('ALERT_EMAIL_TO', os.environ.get('ALERT_EMAIL_TO', os.environ.get('GMAIL_USER', '')))
os.environ.setdefault('GROQ_API_KEY', os.environ.get('GROQ_API_KEY', ''))

ok = bool(os.environ.get('TELEGRAM_BOT_TOKEN') and os.environ.get('TELEGRAM_CHAT_ID') and os.environ.get('GMAIL_USER') and os.environ.get('GMAIL_APP_PASSWORD')); print('Secrets:', 'OK' if ok else 'MISSING – add Colab Secrets or paste above, then re-run.')

## 3. Mount Google Drive (for DB backups + persistence)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.insert(0, str(MNEMOS_ROOT))
from engine.orchestrator import set_drive_mount
set_drive_mount(Path('/content/drive/MyDrive'))
print('Drive mounted. Backups will go to Drive.')

## 4. One-click: Run MNEMOS 24/7

In [ ]:
import sys
sys.path.insert(0, str(MNEMOS_ROOT))

from engine.orchestrator import run_forever
from storage.db import init_db

init_db()
print('MNEMOS 2.1 started. Heartbeats below; alerts to Telegram/Email. Daily heartbeat + weekly/monthly reports run automatically.')
print('To stop: Runtime -> Interrupt execution.')
# Use run_forever without supervisor so Colab does not restart the kernel on exception
run_forever(backup_interval_ticks=20, daily_task_interval_ticks=60)